In [1]:
import pandas as pd
import numpy as np
import nltk
import collections

In [28]:
temp = pd.read_csv("cp2077_reviews_cleaned.csv.zip", compression="zip")

# divide the df_cleaned based on "Recommended or Not Recommended" column
temp_recommended = temp.loc[temp["Recommended or Not Recommended"] == True,:].reset_index(drop=True)
temp_not_recommended = temp.loc[temp["Recommended or Not Recommended"] == False,:].reset_index(drop=True)

# divide temp_recommended into train and test using random sample seed
temp_recommended_train = temp_recommended.sample(frac=0.8, random_state=1).reset_index(drop=True)
temp_recommended_test = temp_recommended.drop(temp_recommended_train.index).reset_index(drop=True)

# divide temp_not_recommended into train and test using random sample seed
temp_not_recommended_train = temp_not_recommended.sample(frac=0.8, random_state=1).reset_index(drop=True)
temp_not_recommended_test = temp_not_recommended.drop(temp_not_recommended_train.index).reset_index(drop=True)

# Combine test dataset
temp_test = pd.concat([temp_recommended_test, temp_not_recommended_test], axis=0).reset_index(drop=True)

In [34]:
temp_recommended_train['Review'] = temp_recommended_train['Review'].astype(str)
temp_not_recommended_train['Review'] = temp_not_recommended_train['Review'].astype(str)

In [52]:
bow_recom = collections.Counter([y for x in temp_recommended_train.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in temp_not_recommended_train.Review for y in x.split()])

In [53]:
# bow_recom to pandas dataframe
df_recom = pd.DataFrame.from_dict(bow_recom, orient='index').reset_index()
df_recom.columns = ['Word', 'Count']
df_recom = df_recom.sort_values(by='Count', ascending=False).reset_index(drop=True)

# bow_not_recom to pandas dataframe
df_not_recom = pd.DataFrame.from_dict(bow_not_recom, orient='index').reset_index()
df_not_recom.columns = ['Word', 'Count']
df_not_recom = df_not_recom.sort_values(by='Count', ascending=False).reset_index(drop=True)


In [19]:
# df_recom = pd.read_csv("bow_recom.csv")
# df_not_recom = pd.read_csv("bow_not_recom.csv")

# # split df_recom into train and test
# train_recom = df_recom.sample(frac=0.8, random_state=200).reset_index(drop=True)
# test_recom = df_recom.drop(train_recom.index).reset_index(drop=True)
# print(f"train size: {len(train_recom)}, test size: {len(test_recom)}")

# # split df_not_recom into train and test
# train_not_recom = df_not_recom.sample(frac=0.8, random_state=200).reset_index(drop=True)
# test_not_recom = df_not_recom.drop(train_not_recom.index).reset_index(drop=True)
# print(f"train size: {len(train_not_recom)}, test size: {len(test_not_recom)}")

# # combine test_recom and test_not_recom
# test = pd.concat([test_recom, test_not_recom], axis=0).reset_index(drop=True)

train size: 22989, test size: 5747
train size: 27086, test size: 6771


## Combining Recommended and Not Recommended BoW

In [13]:
# combined = pd.read_csv("cp2077_reviews_cleaned.csv.zip", compression="zip")

# # drop column Date Timestamp Created
# combined = combined.drop(columns=["Date Timestamp Created"])

# # make sure reviews are strings
# combined["Review"] = combined["Review"].astype("str")

# # split combined into train and test
# train = combined.sample(frac=0.8, random_state=200).reset_index(drop=True)
# test = combined.drop(train.index).reset_index(drop=True)
# print(f"train size: {len(train)}, test size: {len(test)}")

train size: 8400, test size: 2100


## Testing the code for calculating weight

In [65]:
love = df_recom.loc[df_recom["Word"]=="hate",:]
love1 = df_not_recom.loc[df_not_recom["Word"]=="hate",:]
print(love)
print(love1)

     Word  Count
408  hate    141
     Word  Count
738  hate     99


In [67]:
a = love['Count'].item()
b = love1['Count'].item()
sum1 = df_recom['Count'].sum()
sum2 = df_not_recom['Count'].sum()

# adore = 60 (+) 3 (-) 1000  || 63
the_tfidf2_pos = np.log((a + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf2_pos)

the_tfidf3_neg = np.log((b + 1)) * np.log((sum1+sum2)/(a+b))
print(the_tfidf3_neg)

41.0223618202121
38.1197639500956


In [58]:
def define_sentiment(row):
    rec = 0
    not_rec = 0
    sum1 = df_recom['Count'].sum()
    sum2 = df_not_recom['Count'].sum()
    for i in row:
        if i in df_recom["Word"].values:
            a = df_recom.loc[df_recom["Word"]==i, "Count"].item()
        else:
            a = 1
        if i in df_not_recom["Word"].values:
            b = df_not_recom.loc[df_not_recom["Word"]==i, "Count"].item()
        else:
            b = 1
        rec += np.log((a + 1)) * np.log((sum1+sum2)/(a+b))
        not_rec += np.log((b + 1)) * np.log((sum1+sum2)/(a+b))
    if rec > not_rec:
        return True
    else:
        return False

In [64]:
# limit temp_test to 10 rows
temp_test_1 = temp_test.sample(10)

temp_test_1["Sentiment"] = temp_test_1["Review"].apply(lambda x: define_sentiment(x.split()))
temp_test_1

,Review,Recommended or Not Recommended,Date Timestamp Created,Sentiment
2051,two crashes and several game breaking bugs tha...,False,1607574052,False
108,damn this went from blowing things with keanu ...,True,1607728716,False
561,think the game great very immersive gives the ...,True,1610925240,False
375,when works well its fantastic game full conten...,True,1615158235,False
615,tldr some people get paid throw shit some are ...,True,1613521715,False
1926,despite enjoying the game quite bit the amount...,False,1609963480,False
401,amazing game haunted sensationalist media cove...,True,1614382110,False
1451,the game okay have had fun playing but would s...,False,1610086669,False
523,absolutely slays best rpg lifetime,True,1608310632,True
1174,got hours the game mainly because wanted much ...,False,1608098113,False
